# Film Recommendation: Algorithm

Two approaches

- Fist one is the original one (with the best performances from the algorithm description task)
- Second one is with DFS as were recommended

## First approach

In [ ]:
from collections import defaultdict


def film_adviser(movies, similarities, friends):
    if not movies:
        return None

    # define the max value
    # -1 is chosen instead of -inf
    # do not need to import extra libraries
    max_value = -1

    # define the return variable
    advised_film = None

    # define the film dictionary with default values
    # the dictionary keeps all data about each film (database)
    film_dict = {film: {'Score': 0,
                        'Discussability': 0,
                        'Uniqueness': 0,
                        'Seen_similarities': 0
                        }
                 for film in movies}

    # fill the dictionary
    # discussability is the number of friends of user,
    # who have already seen that movie
    for film in sum(friends, []):
        film_dict[film]['Discussability'] += 1

    # get the same component list

    # define a parent child dictionary
    parent_child_dict = dict()

    # find a parent for each film
    def parent(film):
        if parent_child_dict[film] == film:
            return film
        parent_child_dict[film] = parent(parent_child_dict[film])
        return parent_child_dict[film]

    # add the data in the dictionary
    # parent child dictionary
    for film1, film2 in similarities:
        if film1 not in parent_child_dict:
            parent_child_dict[film1] = film1
        if film2 not in parent_child_dict:
            parent_child_dict[film2] = film2
        parent_child_dict[parent(film1)] = parent(film2)

    # define the dictionary to join the data in component
    # film : {film, other film from the same component}
    same_component_dict = defaultdict(set)
    for film1 in parent_child_dict.keys():
        same_component_dict[parent(film1)].add(film1)

    # list of components = values from the dictionary
    same_component_list = list(same_component_dict.values())

    # if no same component than all score are 0
    # to return random movie change type to set and back to list
    # and return the first element
    # set is not sorted, return will be different
    if not same_component_list:
        return list(set(movies))[0]

    # calculate the uniqueness and the score
    for component in same_component_list:
        # number of elements in component
        n = len(component)
        # define the variable to find all numbers of friends
        # who seen movies in selected component
        all_seen = sum([film_dict.get(film)['Discussability'] for film in component])

        # add the data in the dictionary
        # for each film minus the number of friends who seen selected film
        # (define the number of friends who seen similar films)
        for film in component:
            film_dict.get(film)['Seen_similarities'] = all_seen - film_dict.get(film)['Discussability']

            # uniqueness is 1 divided by the mean number of similar movies
            # that the user's friends have already seen
            #  1/mean = 1 / (seen_similarities/len(similarities)) =
            # = len(similarities) / seen_similarities

            # seen_similarities != 0 because we can't divide by 0
            if film_dict.get(film)['Seen_similarities'] != 0:
                uniqueness = (n - 1) / film_dict.get(film)['Seen_similarities']
                # add the data in the dictionary
                film_dict[film]['Uniqueness'] = uniqueness

            # calculate the score
            # score = discussability * uniqueness
            score = film_dict.get(film)['Discussability'] * film_dict.get(film)['Uniqueness']
            #  add the data in the dictionary
            film_dict[film]['Score'] = score

            # check the current value with max value
            if score > max_value:
                max_value = score
                # keep the film with max value
                advised_film = film

    # return the film with max value as recommended
    return advised_film

if __name__ == '__main__':
    movies = ["Parasite", "1917", "Ford v Ferrari", "Jojo Rabbit", "Joker"]

    similarities = [["Parasite", "1917"],
                    ["Parasite", "Jojo Rabbit"],
                    ["Joker", "Ford v Ferrari"]]

    friends = [["Joker"],
               ["Joker", "1917"],
               ["Joker"],
               ["Parasite"],
               ["1917"],
               ["Jojo Rabbit", "Joker"]]

    print('Recommended movie first approach:', film_adviser(movies, similarities, friends))


## Second approach with DFS

In [ ]:
def film_adviser_dfs(movies, similarities, friends):
    if not movies:
        return None

    max_value = -1
    advised_film = None

    film_dict = {film: {'Score': 0,
                        'Discussability': 0,
                        'Uniqueness': 0,
                        'Seen_similarities': 0
                        }
                 for film in movies}

    for film in sum(friends, []):
        film_dict[film]['Discussability'] += 1

    # create a same component list

    # convert similarities to the dictionary
    # contain only current pairs from the similarities list
    # do not have the pairs regarding the transitivity rule
    sim_list_to_dict = {film: set() for film in movies}

    # fill with data from similarities list
    for film_pair in similarities:
        film1 = film_pair[0]
        film2 = film_pair[1]

        sim_list_to_dict[film1].add(film2)
        sim_list_to_dict[film2].add(film1)

    # store the visited films
    visited = {}
    same_component_list = []

    # define DFS algorithm
    def dfs(film, visited, similirar_film, sim_dict):
        visited[film] = True
        similirar_film.add(film)

        for adjacency_film in sim_dict[film]:
            if adjacency_film not in visited:
                dfs(adjacency_film, visited, similirar_film, sim_dict)

        return similirar_film

    # run the DFS
    for film in movies:
        if film not in visited:
            visited[film] = True
            connected_films = dfs(film, visited, set(), sim_list_to_dict)
            same_component_list.append(connected_films)

    if not same_component_list:
        return list(set(movies))[0]

    for component in same_component_list:
        n = len(component)
        all_seen = sum([film_dict.get(film)['Discussability'] for film in component])

        for film in component:
            film_dict.get(film)['Seen_similarities'] = all_seen - film_dict.get(film)['Discussability']

            if film_dict.get(film)['Seen_similarities'] != 0:
                uniqueness = (n - 1) / film_dict.get(film)['Seen_similarities']
                # add the data in the dictionary
                film_dict[film]['Uniqueness'] = uniqueness

            score = film_dict.get(film)['Discussability'] * film_dict.get(film)['Uniqueness']
            film_dict[film]['Score'] = score

            if score > max_value:
                max_value = score
                advised_film = film

    return advised_film


if __name__ == '__main__':
    movies = ["Parasite", "1917", "Ford v Ferrari", "Jojo Rabbit", "Joker"]

    similarities = [["Parasite", "1917"],
                    ["Parasite", "Jojo Rabbit"],
                    ["Joker", "Ford v Ferrari"]]

    friends = [["Joker"],
               ["Joker", "1917"],
               ["Joker"],
               ["Parasite"],
               ["1917"],
               ["Jojo Rabbit", "Joker"]]

    print('Recommended movie second approach (dfs):', film_adviser_dfs(movies, similarities, friends))

## Film Recommendation: Algorithm

### PART 1

#### Question 1
You are tasked with developing a movie recommendation system. You are given a list of movies (their names) and a list of similarities between movies (pairs of movies that are similar). You are also given a list of user's friends and for each friend a list of movies that he has already seen.
Your system should recommend one movie with the highest discussability and uniqueness. Discussability is the number of friends of user, who have already seen that movie. Uniqueness is 1 divided by the mean number of similar movies that the user's friends have already seen. So you should return the film with the highest number: F / S, where F = number of friends who have seen this movie, and S = mean of the number of similar movies seen for each friend. Exclude the movies with S = 0.
If (a, b) and (b, c) are pairs of similar movies, then (a, c) is a pair of similar movies too. Each movie is not counted in its Uniqueness.
Input example. Basically it is up to you to come up with data structure you like or you think easy to work with. In a nutshell you have as an input these parameters (they can be in form of a list/dict etc):

#### Answer:



Short description of the algorithm:

1. Define a default dictionary to store calculated data.
2. Calculate the discussability.
3. Create a list of similar movie components.
4. If there are no components, return a random movie.
5. For each movie in the component, calculate a number of seen similar movies, uniqueness, score.
6. Identify a movie for recommendation.
7. Return the recomended movie.


Detailed description of the algorithm:

0. Define variables for calculations:

max_value = -1 — will be used to compare with the calculeted score to define a recomended movie.

-1 is chosen as an alternative to -inf and no additional libraries needed.

advised_film = None — a returned variable with defined recomendation, by default is None.



1. Define a default dictionary to store calculated data:

key is a movie from movies

value is {'Score': 0,   'Discussability': 0, 'Uniqueness': 0, 'Seen_similarities': 0}


2. Calculate the discussability:

It is defined as 0 by default.

For each movie in sum(friends, []) the dictionary add 1 to Discussability:
film_dict[film]['Discussability'] += 1


3. create a list of similar movie components

Two approaches.


FIRST APPROACH — build the same component list as a result of sets intersection (from the similarities)

Change the lists in similarities to sets.

Check the length of similarities. If it is < 2, define the same components list as similarities.

Else run the intersection process:

- run a loop for i, set1 in enumerate(similarities).
- each set2  in similarities[i + 1:] intersect with set1.
- if intersection is not empy, update set2 with set1, clear the data in set1, redefine set1 as set2.
- define the same component list as list of not empty elements in similarities.


SECOND APPROACH — using recursion and parent-child dictionary

Define parent-child dictionary.

Define a recursive function that links movies.

def parent(film):
    if parent_child_dict[film] == film:
        return film
    parent_child_dict[film] = parent(parent_child_dict[film])
    return parent_child_dict[film]

Consider each pair of movies in the list of similar movies and add them to the dictionary if they are not there: 
parent_child_dict[film1] = film1
parent_child_dict[film2] = film2

-Link movies together: 
parent_child_dict[parent(film1)] = parent(film2)

Define the dictionary to join the data in component: film : {film, other film from the same component}:
for film1 in parent_child_dict.keys():
    same_component_dict[parent(film1)].add(film1)

Define the list of components of similar films as a list of values of the constructed dictionary.


4. If there are no components, return a random movie:

If there are no the same component, than all scores are 0 (by default).

To return a random movie a type of movies is changed to set and back to list and the first element is returned. Set is not sorted and return will be different.

random library can be used as alterenative.


5. For each movie in the component, calculate a number of seen similar movies, uniqueness, score:

Run a loop for each component in the same component list.

Define n = len(component).

Calculate the number of friends, who seen the movies in selected component:
all_seen = sum([film_dict.get(film)['Discussability'] for film in component])


Run a loop for each movie in selected component (*):

Calculate the number of seen similar movies for each movie:
all_seen — film_dict.get(film)['Discussability'].

Calculate uniqueness is 1 divided by the mean number of similar movies that the user's friends have already seen. Simplify the formula as 1/mean = 1 / (seen_similarities/len(similarities)) = len(similarities) / seen_similarities.

If the number of seen similar movies is 0, than there are no calculations, uniqueness = score = 0 by default.

If the number of seen similar movies is not 0, calculate uniqueness(n-1 because n is the number of all movies in selected component):
uniqueness = (n - 1) / film_dict.get(film)['Seen_similarities']

Add the data in the dictionary.

Calculate the score as discussability * uniqueness, take the data from the dictionary:
score = film_dict.get(film)['Discussability'] * film_dict.get(film)['Uniqueness']

Add the data in the dictionary.


6. Identify a movie for recommendation:

In the loop (*) check the current score with max value.

If the current score is bigger, update the max value and redefine advised_film with current film.


7. Return the recomended movie:

return advised_film



2. Please provide the proof that your algorithm will come up with correct/optimal solution.

There are no undefined values in the program code. All calculated data are defined as 0 and are further updated during the program.

The program runs from start to finish without interruption from calculation to calculation.

Let's take a closer look.

The first loop computes Discussability. 

- The calculation is iterated over the movies in friends and 1 is added to the value of Discussability (default is 0) each time a movie is found in the list.
- The default value is definitely 0, so there should be no errors in the calculation.
- The movie is already in the dictionary, so updating the dictionary data by key will not lead to an error.
- The condition of the task (on the forum) says that there are no empty entries, that is, the list of films is not empty (if it is empty, then we can check it before the cycle and return None).
- If none of the friends have seen any movies, that is, friends list is empty, then there will be no calculations and the default values will remain, defined as zero in the dictionary.


Finding the same compnent list.

FIRST APPROACH

- If there are only 0 or 1 elements in the list, then the list of components is the same as the list of similar movies and the process of set intersection is not started.
- Otherwise, the process of set intersection stars (described above). The data is updated according to the described algorithm, there should be no errors during the launch process.

SECOND APPROACH

- All pairs of similar films are considered.
- The connections between the movies are built correctly.
- A built list of components of similar movies is created.


Updating the dictionary data (calculations).

- If there are no any components (all movies ado not have similar movies) we do not need to recalculate the data (uniqueness, score), they will be 0 (it is already defined). We just need to return a random movies (described above). No errors shold be here.
- If there are components, then each movie from the component is considered in further calculations.
- If a movie is not in the components, then it has no similar movies and its values for uniqueness and score are zero (defined by default).
- To calculate the uniqueness, the moment of division by zero is taken into account.
- Updating data in the dictionary by key will work, since all movies have already been previously added to the dictionary.
- Formulas for calculating uniqueness and score are defined correctly.


Finding the recomended movie.

- The check goes in the value calculation cycle.
- The maximum value is initially defined as -1. In this case, this is sufficient, since all calculated values >= 0.
- If the current score is greater than the maximum, then the value of the maximum is updated and the current movie for this maximum is stored in the variable advised_film, which is returned upon completion of the process of calculating data for all movies.


Conclusion:

- During the program, all input data were used to build an algorithm in accordance with the conditions of the task.
- All formulas are defined correctly.
- Calculations for each movie are done.
- Moments of errors when using formulas (empty entries, division by zero) are taken into account.



 
#### Question 3
What is the time complexity of your algorithm? Please provide proof for it.

FOR THE FIRST APPROACH

Let us define the following variables:
M = len(movies), S = len(similarities), F = len(friends)
- build a dictionary of movies: we have M movies and 4 variables for each movie (Score, Discussability, Uniqueness, Seen_similarities) → O(M)
- calculate Discussability: for film in sum(friends, []), we have F friends and each senn maximum M movies (worst case) → O(F*M)
- similarities: similarities = [set(element) for element in similarities if element], S pairs in the list → O(S)
- for i, set1 in enumerate(similarities) →  O(S)
- nested loop: for set2 in similarities[i + 1:] → O(S) → for both (S^2)
- same_component_list = [element for element in similarities if element] → O(S)
- for component in same_component_list: O(len(component_list)), in worst case is equal to number of movies → O(M)
-- nested loop: for film in component, in worst case → O(M) → for both O(M^2)

In the worst case → O(F*M + S^2 + M^2).

FOR THE SECON APPROACH

- build a dictionary of movies: we have M movies and 4 variables for each movie (Score, Discussability, Uniqueness, Seen_similarities) → O(M)
- calculate Discussability: for film in sum(friends, []), we have F friends and each senn maximum M movies (worst case) → O(F*M)
- for film1, film2 in similarities → O(S)
– nested parent_child_dict[parent(film1)] = parent(film2) → O(M) → for both O(S+M)
- for film1 in parent_child_dict.keys() → O(M)
- same_component_list = [x for x in same_component_dict.values()] → in worst case → O(M)
- for component in same_component_list: O(len(component_list)), in worst case is equal to number of movies → O(M)
-- nested loop: for film in component, in worst case → O(M) → for both O(M^2)

In the worst case → O(F*M + S  + M^2).
In worst case S = M * (M — 1) / 2 → O(S) is equal to O(M^2).
We get O(F*M + M^2).


#### Question 4
What is the memory complexity of your algorithm? Please provide proof for it.

When working with big data, this option is always the loser. Especially in a situation where the calculated data must also be stored.

FOR THE FIRST APPROACH

- define a movies dictionary → O(M)
- sum(friends, []) - > in worst case O(F*M)
- similarities → O(S)
- enumerate(similarities) → O(S)
- similarities[i + 1:] - > O(S)
- redefined similarities: in worsed case → O(S*F)
- same_component_list: in worst case → O(M)

We get O(F*M + S*F + S + M) = O(F*M + F*S).

In worst case S = M * (M — 1) / 2 → O(S) is equal to O(M^2).

We get O(F * M^2).



FOR THE SECOND APROACH

- define a movies dictionary → O(M)
- sum(friends, []) - > in worst case O(F*M)
- parent_child_dict → O(M)
- similarities → O(S)
same_component_dict → in worst case → O(M)
- same_component_list: in worst case → O(M)

We get O(F*M + S + M).

In worst case S = M * (M — 1) / 2 → O(S) is equal to O(M^2).

We get O(F*M + M^2).